In [ ]:
# !pip install PyYAML

In [1]:
import cv2
import numpy as np
import yaml
from yaml.loader import SafeLoader

## Load Yaml file

In [3]:
with open('data.yaml','r') as f:
    data_yaml=yaml.load(f,Loader=SafeLoader)

names=data_yaml['names']
print(names)

['fox', 'skunk', 'racoon', 'cow', 'chicken', 'person', 'horse', 'cat', 'dog', 'goat']


## Load Model

In [10]:
model=cv2.dnn.readNetFromONNX('./model/weights/best.onnx')

# customize accordingly
model.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
model.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

## Make Predictions

In [349]:
# Load Image
image=cv2.imread('./test.jpg')
img=image.copy()

# Convert Image to square Image
r,c,d=img.shape
max_wd=max(r,c)
sqr_img=np.zeros((max_wd,max_wd,3),dtype=np.uint8) # make sure this dtype is uint8 and not int8
sqr_img[0:r,0:c]=img

cv2.imshow('test-img',sqr_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# # Make Preds using yolo on sqr_img
YOLO_WH=640 # width and height of image require by our yolo model

blob=cv2.dnn.blobFromImage(sqr_img,1/255,(YOLO_WH,YOLO_WH),swapRB=True,crop=False) 
model.setInput(blob)
pred=model.forward()

In [350]:
print(pred.shape) # -> signifies yolo (N,num_bounding_box_detected,preds_per_bndbox)

(1, 25200, 15)


In [351]:
## Do a Non maximal suppression
detections=pred[0]
classes=[]
confidence_scores=[]
bnd_boxes=[]

# x_factor,y_factor to map predicted bndbox on original image
input_img_width,input_img_height,_=sqr_img.shape
x_factor=input_img_width/YOLO_WH
y_factor=input_img_height/YOLO_WH

# Set Threshold for confidence_score and cls_probability (the more it is the more robust ur model will be but the less objects might get detected)
confidence_threshold=0.5
class_threshold=0.25

for detection in detections:
    confidence_score=detection[5]
    if confidence_score>confidence_threshold:
        class_score=detection[5:].max()
        class_id=detection[5:].argmax()
        if class_score>class_threshold:
            # Extract bndbox info
            cx,cy,w,h=detection[:4]
            
            # Find left,top,width and height of bndbox that will overlay over orgnl_img
            left=int((cx-0.5*w)*x_factor)
            top=int((cy-0.5*h)*y_factor)
            width=int(w*x_factor)
            height=int(h*y_factor)

            box=np.array([left,top,width,height])
            
            bnd_boxes.append(box)
            classes.append(class_id)
            confidence_scores.append(confidence_score)

# clean
bnd_boxes=np.array(bnd_boxes).tolist()
confidence_scores=np.array(confidence_scores).tolist()
classes=np.array(classes).tolist()

In [354]:
# Do non-maximum-supression
indexes=cv2.dnn.NMSBoxes(bnd_boxes,confidence_scores,confidence_threshold,0.5).flatten()
indexes

array([2], dtype=int32)

In [355]:
##### Draw bndbox
temp_img=sqr_img.copy()
for idx in indexes:
    # extract bnd_box
    x,y,w,h=bnd_boxes[idx]
    confidence_score=int(confidence_scores[idx]*100)
    cls_id=classes[idx]
    class_name=names[cls_id]

    text=f'{class_name}:{confidence_score}%'

    cv2.rectangle(temp_img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(temp_img,(x,y-30),(x+w,y),(255,255,255),-1)
    cv2.putText(temp_img,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)

In [356]:
cv2.imshow('pred',temp_img)
cv2.imshow('original',sqr_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [357]:
cv2.imwrite('test-pred',temp_img)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:708: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'cv::imwrite_'
